In [ ]:
import time
import random
import pandas as pd
import numpy as np
import copy
import math
import tensorflow
import sklearn

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

In [ ]:
import random

def generate_random_individuals(num_individuals, num_features, max_features=None, verbose=False):
    """
    Randomly generates individuals
    The number of individuals to generate is given by the num_individuals parameter
    The length of each individual is equal to the num_features parameter
    The maximum number of active features for every individual is given by the max_features parameter
    """
    if verbose: print('GENERATING RANDOM INDIVIDUALS.... ')

    individuals = list()

    for _ in range(num_individuals):
        individual = ''
        for col in range(num_features):
            # For each char in the individual, a 1 or a 0 is randomly generated
            if individual.count('1') == max_features:
                individual += '0'
                continue

            individual += str(random.randint(0, 1))

        if verbose: print(f'Genrated a new indivudal: {individual}')
        individuals.append(individual)

    if verbose: print(f'Generated list of {num_individuals} individuals: {individuals}')

    return individuals

In [ ]:
def create_model(X, verbose=False):
    """
    X: training dataset to be used. Its shape is used to set the input shape for the model.
    """

    model = Sequential()

    model.add(Dense(32, input_dim=X.shape[1], activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    if verbose: print('MODEL SUMMARY: \n')
    if verbose: print(model.summary())

    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

    return model

In [ ]:
def get_fitness_func(individual, dataframe, verbose=False):
    """
    Calculate accuracy for the individual passed as parameter.
    Both the dataframe and the y_data parameters are used for training and evaluating the model.
    """
    if verbose: print('Calculating accuracy for individual ', individual)

    # generate_dataframes_for_training() function splits a given dataset into training and test data,
    # and separates labels and rest of features
    X_train, X_test, y_train, y_test = generate_dataframes_for_training(dataframe)

    X_train = X_train.loc[:, [True if individual[i] == '1' else False for i in range(len(individual))]]
    X_test = X_test.loc[:, [True if individual[i] == '1' else False for i in range(len(individual))]]

    model = create_model(X_train, individual)

    X_train = np.asarray(X_train).astype(np.float64)
    X_test = np.asarray(X_test).astype(np.float64)
    y_train = np.asarray(y_train).astype(np.float64)
    y_test = np.asarray(y_test).astype(np.float64)

    model.fit(X_train, y_train, epochs=100, verbose=1 if verbose else 0)

    pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, pred.round())
    if verbose: print(f"Accuracy for the classifier trained for individual {individual}: ", accuracy)

    return accuracy

In [ ]:
def get_weights(population):
    """
    Calculate weights from the population filled with the accuracies
    """
    total_accuracies = 0
    new_population = []

    # Get the sum of all accuracies of the population
    for individual in population:
        total_accuracies += individual[1]

    # For each individual, calculate its weight by dividing its accuracy by the overall sum calculated above
    for individual in population:
        weight = individual[1]/total_accuracies
        # Store the individual and its weight in the final population list
        new_population.append((individual[0], float(weight*100)))

    return new_population

In [ ]:
def fill_population(individuals, dataframe, y_data, verbose=False):
    """
    Fills the population list with individuals and their weights
    """
    population = list()

    for individual in individuals:

        # Get the value of the fitness function (accuracy of the model)
        if verbose: print(f'Calculating fitness function value for individual {individual}')
        accuracy = get_fitness_func(individual, dataframe, verbose)

        # Check that the value is not the goal state (in this case, an accuracy of 80% is a terminal state)
        if float(accuracy) > 0.8:
            if verbose: print(f'Goal state found for individual {individual}')
            return individual

        individual_complete = (individual, accuracy)
        population.append(individual_complete)

    # The final population list is created, which contains each individual together with its weight
    # (weights will be used in the reproduction step)
    new_population = get_weights(population)
    if verbose: print(f'Generated population list (with weights): {new_population}')

    return new_population

In [ ]:
import copy

def choose_parents(population, counter):
    """
    From the population, weighting the probabilities of an individual being chosen via the fitness
    function, takes randomly two individual to reproduce
    Population is a list of tuples, where the first element is the individual and the second
    one is the probability associated to it.
    To avoid generating repeated individuals, 'counter' parameter is used to pick parents in different ways, thus
    generating different individuals
    """
    # Pick random parent Number 1 and Number 2
    # (get_n_individual() function randomly picks an individual following the distribution of the weights)
    if counter == 0:
        parent_1 = get_n_individual(0, population)
        parent_2 = get_n_individual(1, population)
    elif counter == 1:
        parent_1 = get_n_individual(0, population)
        parent_2 = get_n_individual(2, population)

    else:
        probabilities = (individual[1] for individual in population)
        individuals = [individual[0] for individual in population]
        parent_1, parent_2 = random.choices(individuals, weights=probabilities, k=2)

    return [parent_1, parent_2]


def mutate(child, prob=0.1):
    """
    Randomly mutates an individual according to the probability given by prob parameter
    """
    new_child = copy.deepcopy(child)
    for i, char in enumerate(new_child):
        if random.random() < prob:
            new_value = '1' if char == '0' else '0'
            new_child = new_child[:i] + new_value + new_child[i+1:]

    return new_child


def reproduce(individual_1, individual_2):
    """
    Takes 2 individuals, and combines their information based on a
    randomly chosen crosspoint.
    Each reproduction returns 2 new individuals
    """
    # Randomly generate a integer between 1 and the length of the individuals minus one, which will be the crosspoint
    crosspoint = random.randint(1, len(individual_1)-1)
    child_1 = individual_1[:crosspoint] + individual_2[crosspoint:]
    child_2 = individual_2[:crosspoint] + individual_1[crosspoint:]
    child_1, child_2 = mutate(child_1), mutate(child_2)

    return [child_1, child_2]


def generation_ahead(population, verbose=False):
    """
    Reproduces all the steps for choosing parents and making
    childs, which means creating a new generation to iterate with
    """
    new_population = list()

    for _ in range(int(len(population)//2)):
        # According to the weights calculated before, choose a set of parents to reproduce
        parents = choose_parents(population, counter=_)
        if verbose: print(f'Parents chosen: {parents}')

        # Reproduce the pair of individuals chose above to generate two new individuals
        childs = reproduce(parents[0], parents[1])
        if verbose: print(f'Generated children: {childs}\n')
        new_population += childs

    return new_population

In [ ]:
def main_loop(ind_num, dataframe, max_iter=3, verbose=False):
    """
    Performs all the steps of the Genetic Algorithm
    1. Generate random population
    2. Fill population with the weights of each individual
    3. Check if the goal state is reached
    4. Reproduce the population, and create a new generation
    5. Repeat process until termination condition is met
    """
    # Generate individuals (returns a list of strings, where each str represents an individual)
    individuals = generate_random_individuals(ind_num, 65, verbose=verbose)

    # Returns a list of tuples, where each tuple represents an individual and its weight
    population = fill_population(individuals, dataframe, verbose)

    # Check if a goal state is reached
    # When goal state is reached, fill_population() function returns a str, otherwise continue
    if isinstance(population, str):
        return population

    # Reproduce current generation to generate a better new one
    new_generation = generation_ahead(population, verbose)

    # After the new generation is generated, the loop goes on until a solution is found or until the maximum number of
    # iterations are reached
    iteration_count = 0
    while iteration_count < max_iter:
        if verbose: print(f'\n\n\nITERATION NUMBER {iteration_count+1} (Iteration max = {max_iter+1})\n\n\n')
        population = fill_population(new_generation, dataframe, verbose)

        # Check if a goal state is reached
        if isinstance(population, str):
            break

        new_generation = generation_ahead(population, verbose)
        iteration_count += 1

    return population

In [ ]:
def train_with_original_data(dataframe):
  """
  dataframe: normalized and preprocessed dataset, ready for training
  max_ind: individual with highest accuracy obtained from the execution of the GA algorithm
  """
  # Create the model
  model = Sequential()
  model.add(Dense(64, input_dim=dataframe.shape[1], activation='relu'))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

  start = time.time()
  model.fit(X_train, y_train, epochs=128, verbose=1)
  print('Time elapsed for training the model with the full dataset: ', time.time() - start, ' seconds')

def evaluate_model(model, X_test, y_test):
  """
  Print the metrics of the model's performance (Binary Classifier)
  """
  pred = model.predict(X_test)
  accuracy = accuracy_score(y_test, pred.round())
  print("Accuracy for the classifier: ", accuracy)
  print('Recall: ', recall_fun(y_test, pred.round()))
  print('Precision: ', precision_fun(y_test, pred.round()))
  print('F1 Score: ', f1_fun(y_test, pred.round()))


In [ ]:

def train_with_optimized_data(optimized_features_dataframe, y_data, max_ind):
  """
  optimized_features_dataframe: normalized and preprocessed dataset, without labels
  y_data: array of labels to be predicted. Can be extracted from the original dataframe
  by taking the 'Churn' column
  max_ind: individual with highest accuracy obtained from the execution of the GA algorithm
  """
  optimized_dataframe = optimized_features_dataframe.loc[:, [True if char == '1' else False for char in max_ind+'0']]

  # Add the label to predict to the optimized dataset
  optimized_dataframe['Churn'] = y_data

  # Create the model
  model = Sequential()
  model.add(Dense(64, input_dim=optimized_dataframe.shape[1], activation='relu'))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

  # Generate dataframes for both training and testing
  X_train, X_test, y_train, y_test = generate_dataframes_for_training(optimized_dataframe)
  X_train = np.asarray(X_train).astype('float32')
  X_test = np.asarray(X_test).astype('float32')
  y_train = np.asarray(y_train).astype('float32')
  y_test = np.asarray(y_test).astype('float32')

  # Train the model
  start = time.time()
  model.fit(X_train, y_train, epochs=128, verbose=1)
  print('Time elapsed for training the model with the optimized dataset: ', time.time() - start, ' seconds')